In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from pathlib import Path

from sen3r.sen3r import Core
from sen3r.nc_engine import NcEngine
from sen3r.commons import Utils, Footprinter, DefaultDicts
from sen3r.tsgen import TsGenerator
from pathlib import Path

dd = DefaultDicts()
fp = Footprinter()
tsg = TsGenerator()

## Running SEN3R with a single WFR image

In [ ]:
# TESTING SINGLE IMAGE PROCESSING
args = {}

args['input'] = Path('D:/S3/sen3r_wfr_test/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.SEN3')
args['out'] = Path('D:/sen3r_out/')
args['roi'] = Path('D:/A1_JM/areas/paper_areas/14100000_manacapuru.geojson')
args['product'] = 'wfr'
args

In [ ]:
s3r = Core(args)

In [ ]:
%%time
band_data, img_data, doneList = s3r.build_single_file()

## For a folder of WFR images

In [ ]:
mt_args = {}

mt_args['input'] = Path('D:/S3/sen3r_wfr_test/')
mt_args['out'] = Path('D:/sen3r_out/')
mt_args['roi'] = Path('D:/A1_JM/areas/paper_areas/14100000_manacapuru.geojson')
mt_args['product'] = 'wfr'
mt_args

s3r = Core(mt_args)

In [ ]:
%%time
doneList = s3r.build_intermediary_files()

In [ ]:
doneList

### OGR GML to SHP

In [2]:
# Read the xfdumanifest.xml from the Sentinel-3 image
s3_xml_manifest = Path("D:/S3/sen3r_wfr_test/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.SEN3/xfdumanifest.xml")

In [3]:
# This will create a .gml and a .shp file from the xfdumanifest.xml
xmldict = fp.manifest2shp(s3_xml_manifest,"D:\\sen3r_out\\footprint")

In [4]:
# Test if a user ROI touches the image footprint
foot = 'D:/sen3r_out/footprint.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/12350000_fonte_boa.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/12900001_tefe.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/13100090_coari.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/14100000_manacapuru.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/14910000_negro.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/15860000_borba_madeira.shp'
# roi = 'D:/A1_JM/areas/paper_areas/shp/16030000_itacoatiara.shp'
roi = 'D:/A1_JM/areas/paper_areas/shp/17050001_obidos.shp'

In [5]:
fp.touch_test(footprint_shp=foot, roi_shp=roi)

False

In [1]:
## SEN3R internal plot functions

In [5]:
csv_n1 = Path('d:/sen3r_out/CSV_N1/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.csv')
csv_n2 = Path('d:/sen3r_out/CSV_N2/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.csv')

df_n1 = pd.read_csv(csv_n1)
df_n2 = pd.read_csv(csv_n2)

In [34]:
csv_n1 = Path('d:/sen3r_out/CSV_N1/S3A_OL_2_WFR____20190630T134235_20190630T134535_20190701T223549_0179_046_238_3060_MAR_O_NT_002.csv')
csv_n2 = Path('d:/sen3r_out/CSV_N2/S3A_OL_2_WFR____20190630T134235_20190630T134535_20190701T223549_0179_046_238_3060_MAR_O_NT_002.csv')

df_n1 = pd.read_csv(csv_n1)
df_n2 = pd.read_csv(csv_n2)

In [35]:
tsg.imgdpi=300
figname = '20190630T134235'

In [36]:
tsg.s3l2_custom_reflectance_plot(df=df_n1,
                                 figure_title=f'{figname}\n',
                                 c_lbl='Aer. Optical Thickness (T865)',
                                 save_title='D:/sen3r_out/IMG/20190630T134235_2_raw.png'
                                )

In [37]:
tsg.s3l2_custom_reflectance_plot(df=df_n2,
                                 figure_title=f'{figname}\n',
                                 c_lbl='Aer. Optical Thickness (T865)',
                                 save_title='D:/sen3r_out/IMG/20190630T134235_2_no_clustering.png'
                                )

In [38]:
# Apply DBSCAN
tsg.db_scan(df_n2, dd.clustering_methods['M4'])

# Delete rows classified as noise:
indexNames = df_n2[df_n2['cluster'] == -1].index
df_n2.drop(indexNames, inplace=True)

if len(df_n2) > 1:
    clusters = df_n2.groupby(by='cluster').median()
    k = Utils.find_nearest(clusters['Oa21_reflectance:float'], 0)
    # Delete rows from the other clusters:
    indexNames = df_n2[df_n2['cluster'] != k].index
    df_n2.drop(indexNames, inplace=True)

In [39]:
tsg.s3l2_custom_reflectance_plot(df=df_n2,
                                 figure_title=f'{figname}\n',
                                 c_lbl='Aer. Optical Thickness (T865)',
                                 save_title='D:/sen3r_out/IMG/20190630T134235_2.png'
                                )

In [ ]:
import openpyxl

workbook = openpyxl.load_workbook('D:/sen3r_out/sen3r.xlsx')
worksheet = workbook['wfr']

## Create a dictionary of column names
ColNames = {}
Current  = 0
for COL in worksheet.iter_cols(1, worksheet.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

ColNames
## Now you can access by column name
## (My data has a column named 'Dogs')
# for row_cells in worksheet.iter_rows(min_row=1, max_row=4):
#     print(row_cells[ColNames['Dogs']].value)

In [108]:
ColNames['Quality']

41